In [ ]:
import rioxarray
import xarray
import geopandas
import shapely.geometry
import shapely.ops
import rasterio
import rasterio.mask
import numpy
import matplotlib.pyplot
import scipy.ndimage
import scipy.interpolate
mport scipy.spatial
import pathlib
import pdal
import json

# Setup paths

In [ ]:
base_path = pathlib.Path(r'C:\Users\pearsonra\Documents\data\Wakanae\Small_test_site')
destination_path = pathlib.Path(r'trimmed_data')
boundary_name = pathlib.Path(r'boundary\boundary.shp')
lidar_name = pathlib.Path(r'points.laz')

destination_dem_name = pathlib.Path(r'dem.nc')
destination_bathymetry_name = pathlib.Path(r'offshore_bathymetry')
destination_boundary_name = pathlib.Path(r'boundary')

# Load in data

In [ ]:
boundary = geopandas.read_file(base_path/destination_path/boundary_name)
bathymetry_trimmed = geopandas.read_file(base_path/destination_path/destination_bathymetry_name)
dem_trimmed = rioxarray.rioxarray.open_rasterio(base_path/destination_path/destination_dem_name, masked=True)

### Read in LiDAR with relevant processing
*  Note will need to worry about ensuring we are in the right crs later. I will ignore this for now. In future may be able to use the 'reprojection' option discussed here: https://pdal.io/tutorial/las.html
*  May also want to filter to only keep data within a polygon (i.e. remove LiDAR outside the catchment). It seems filters.crop should do the ticket. https://pdal.io/stages/filters.crop.html#filters-crop
*  May also want to create a polygon of the LiDAR extents to then filter the DEM. It looks like this can be done with filters. hexbin. https://pdal.io/stages/filters.hexbin.html#filters-hexbin

In [ ]:
pdal_pipeline_instructions = [
    {"type":  "readers.las", "filename": str(lidar_file_name)}
    #{"type":"filters.crop", "polygon":"POINT(0 0 0)"} - this may allow for the data to be filtered as loaded boundary.loc[0]
]

pdal_pipeline = pdal.Pipeline(json.dumps(pdal_pipeline_instructions))
pdal_pipeline.execute();

# Set to same projection

In [ ]:
crs = 2193
boundary = boundary.to_crs(crs)
bathymetry_trimmed = bathymetry_trimmed.to_crs(crs)
dem_trimmed.rio.set_crs(crs);

# Remove DEM where LiDAR
Will try get a polygon of hte DEM extents as part of the PDAL processing pipeline. Create a polygon of the LiDAR extents to then filter the DEM. It looks like this can be done with filters. hexbin. https://pdal.io/stages/filters.hexbin.html#filters-hexbin

In [ ]:
metadata=json.loads(pdal_pipeline.get_metadata())
metadata['metadata']['readers.las'];

In [ ]:
pdal_pipeline.arrays[0]['X']

# Look at creating a DEM
There a re a few options here. 
* Look for routine in a library like PDAL
* Code up a simple routine myself using a KDTree or something similar

`tree = scipy.spatial.KDTree(numpy.c_[pdal_pipeline.arrays[0]['X'].ravel(), pdal_pipeline.arrays[0]['X'].ravel()])`

In [ ]:
tree = scipy.spatial.KDTree(numpy.c_[pdal_pipeline.arrays[0]['X'].ravel(), pdal_pipeline.arrays[0]['X'].ravel()])

# View background trimmed data

In [ ]:
f, ax = matplotlib.pyplot.subplots(figsize=(15, 10))

p = ax.pcolormesh(dem_trimmed.x[:],dem_trimmed.y[:],dem_trimmed.values[0],cmap='viridis', shading='auto')
boundary.plot(ax=ax, color='None', edgecolor='lime', linewidth=5)
bathymetry_trimmed.plot(ax=ax, marker='o', color='red', markersize=5)
matplotlib.pyplot.colorbar(p)

ax.set_title("Combined")

# Save out combined points and DEM

In [ ]:
#dem_generated.to_netcdf(base_path/destination_path/destination_dem_name)